# --- Day 11: Plutonian Pebbles ---

The ancient civilization on Pluto was known for its ability to manipulate spacetime, and while The Historians explore their infinite corridors, you've noticed a strange set of physics-defying stones.

At first glance, they seem like normal stones: they're arranged in a perfectly straight line, and each stone has a number engraved on it.

The strange part is that every time you blink, the stones change.

- Sometimes, the number engraved on a stone changes.
- Other times, a stone might split in two, causing all the other stones to shift over a bit to make room in their perfectly straight line.

As you observe them for a while, you find that the stones have a consistent behavior. Every time you blink, the stones each simultaneously change according to the first applicable rule in this list:

1. **If the stone is engraved with the number `0`, it is replaced by a stone engraved with the number `1`.**
2. **If the stone is engraved with a number that has an even number of digits, it is replaced by two stones.**  
   The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone.  
   *(The new numbers don't keep extra leading zeroes: `1000` would become stones `10` and `0`.)*
3. **If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by `2024` is engraved on the new stone.**

No matter how the stones change, their order is preserved, and they stay on their perfectly straight line.

---

### Task

How will the stones evolve if you keep blinking at them? You take a note of the number engraved on each stone in the line (your puzzle input).

---

### Example

If you have an arrangement of five stones engraved with the numbers `0 1 10 99 999` and you blink once, the stones transform as follows:

1. The first stone, `0`, becomes a stone marked `1`.
2. The second stone, `1`, is multiplied by `2024` to become `2024`.
3. The third stone, `10`, is split into a stone marked `1` followed by a stone marked `0`.
4. The fourth stone, `99`, is split into two stones marked `9`.
5. The fifth stone, `999`, is replaced by a stone marked `2021976`.

So, after blinking once, your five stones would become an arrangement of seven stones engraved with the numbers:

`1 2024 1 0 9 9 2021976`

---

Here is a longer example:

**Initial arrangement:**
```
125 17
```

**After 1 blink:**
```
253000 1 7
```

**After 2 blinks:**
```
253 0 2024 14168
```

**After 3 blinks:**
```
512072 1 20 24 28676032
```

**After 4 blinks:**
```
512 72 2024 2 0 2 4 2867 6032
```

**After 5 blinks:**
```
1036288 7 2 20 24 4048 1 4048 8096 28 67 60 32
```

**After 6 blinks:**
```
2097446912 14168 4048 2 0 2 4 40 48 2024 40 48 80 96 2 8 6 7 6 0 3 2
```

In this example, after blinking six times, you would have `22` stones. After blinking `25` times, you would have `55312` stones!

---

### Puzzle

Consider the arrangement of stones in front of you. **How many stones will you have after blinking 25 times?**


In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import itertools


@dataclass
class Rule(ABC):

    @abstractmethod
    def is_applicable(self, stone: int) -> bool:
        pass

    @abstractmethod
    def __call__(self, stone: int) -> list[int]:
        pass


class ReplaceWithOne(Rule):
    def is_applicable(self, stone: int) -> bool:
        return stone == 0

    def __call__(self, stone: int) -> list[int]:
        return [1]


class SplitInTwoIfEvenLen(Rule):
    def is_applicable(self, stone: int) -> bool:
        return len(str(stone)) & 1 == 0  # is even

    def __call__(self, stone: int) -> list[int]:
        number_str = str(stone)
        mid_point = (len(number_str) + 1) // 2
        half_l, half_r = number_str[:mid_point], number_str[mid_point:]
        return [int(half_l), int(half_r)]


class MultiplyBy2024(Rule):
    def is_applicable(self, stone: int) -> bool:
        return True  # default rule (?)

    def __call__(self, stone: int) -> list[Rule]:
        return [stone * 2024]


@dataclass
class StonesLine:
    stones: list[int]

    def __len__(self):
        return len(self.stones)

    def change(self, rules: list[Rule]):
        for i, stone in enumerate(self.stones):
            for rule in rules:
                if rule.is_applicable(stone):
                    self.stones[i] = rule(stone)
                    break
        # flatten stones
        self.stones = list(itertools.chain.from_iterable(self.stones))

    @classmethod
    def from_numbers(cls, numbers: list[int]) -> "StonesLine":
        return cls([int(n) for n in numbers])
    
    @classmethod
    def from_file(cls, filepath: str) -> "StonesLine":
        with open(filepath, "r", encoding="utf-8") as f:
            content = f.read()
        return cls([int(stone) for stone in content.split()])

rules = [ReplaceWithOne(), SplitInTwoIfEvenLen(), MultiplyBy2024()]

stones = StonesLine.from_numbers([0, 1, 10, 99, 999])
n_blinks = 1
for _ in range(n_blinks):
    stones.change(rules=rules)
    print(stones)
len(stones)

StonesLine(stones=[1, 2024, 1, 0, 9, 9, 2021976])


7

In [2]:
stones = StonesLine.from_file("./example.txt")
n_blinks = 25
for i in range(n_blinks):
    stones.change(rules=rules)
    if i+1 == 6:
        print(f"[{i+1}] {len(stones)}: {stones}")
len(stones)

[6] 22: StonesLine(stones=[2097446912, 14168, 4048, 2, 0, 2, 4, 40, 48, 2024, 40, 48, 80, 96, 2, 8, 6, 7, 6, 0, 3, 2])


55312

In [3]:
stones = StonesLine.from_file("./input.txt")
n_blinks = 25
for _ in range(n_blinks):
    stones.change(rules=rules)
len(stones)

197157

# --- Part Two ---
The Historians sure are taking a long time. To be fair, the infinite corridors are very large.

How many stones would you have after blinking a total of 75 times?

In [4]:
# from tqdm.auto import trange

# stones = StonesLine.from_file("./input.txt")
# n_blinks = 75
# for _ in trange(n_blinks):
#     stones.change(rules=rules)
#     print(len(stones))
# len(stones)